# Anatomical Statistics

In [4]:
from brainprint.atlas import Atlas
from brainprint.protocol import Protocol
from brainprint.recon_all.execution_configuration import ExecutionConfiguration
from brainprint.recon_all.results import ReconAllResults
from brainprint.recon_all.differences import ReconAllDifferences
from brainprint.protocol import Protocol
from nilearn import datasets, plotting
from pathlib import Path
from scipy.stats import ks_2samp
from statsmodels.stats.multitest import multipletests
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
pd.options.display.float_format = '{:3.3g}'.format

## Read Results

In [5]:
results = ReconAllResults(
    atlas=Atlas.DESTRIEUX,
    protocol=Protocol.BASE,
    configuration=[
        ExecutionConfiguration.MPRAGE_AND_3T_AND_T2, 
        ExecutionConfiguration.MPRAGE_AND_T2, 
        ExecutionConfiguration.T2, 
        ExecutionConfiguration.DEFAULT,
        ExecutionConfiguration.FLAIR,
        ExecutionConfiguration.MPRAGE_AND_FLAIR,
        ExecutionConfiguration.MPRAGE_AND_3T_AND_FLAIR,
    ],    
    completed_only=True,
    multi_only=True,
    questionnaire_only=False,
)

[2022-08-07 13:12:20,990] Successfully read 5512 recon-all execution results from /home/zvi/Projects/brainprint/data/results.csv.
[2022-08-07 13:12:21,005] Successfully read 5512 recon-all execution configurations from /home/zvi/Projects/brainprint/data/configurations.csv.
[2022-08-07 13:12:21,045] Successfully read 5512 scan research context and metadata from /home/zvi/Projects/brainprint/data/context.csv.
[2022-08-07 13:12:21,480] Filtering 5512 recon-all results
[2022-08-07 13:12:21,482] 3024/5512 recon-all results matching ['The Base (corrected)'] detected.
[2022-08-07 13:12:21,486] 3010/3024 recon-all results matching ['T2 + MPRAGE + 3T', 'T2 + MPRAGE', 'T2', 'Default', 'FLAIR', 'FLAIR + MPRAGE', 'FLAIR + MPRAGE + 3T'] detected.
[2022-08-07 13:12:21,493] Successfully selected 2142 runs from 306 scans with all 7 recon-all execution configuration results.
[2022-08-07 13:12:21,503] 121 subjects with multiple scans found to be associated with 1771 runs from 253 scans.
[2022-08-07 13:1

In [3]:
results.plot_difference_of_means_table()

[2022-08-03 11:41:16,344] 252/1769 recon-all results matching ['T2 + MPRAGE + 3T'] detected.
[2022-08-03 11:41:16,349] 253/1769 recon-all results matching ['Default'] detected.
[2022-08-03 11:41:16,359] 253/1769 recon-all results matching ['T2'] detected.
[2022-08-03 11:41:16,364] 253/1769 recon-all results matching ['Default'] detected.
[2022-08-03 11:41:16,376] 253/1769 recon-all results matching ['FLAIR'] detected.
[2022-08-03 11:41:16,383] 253/1769 recon-all results matching ['Default'] detected.
[2022-08-03 11:41:16,394] 252/1769 recon-all results matching ['FLAIR + MPRAGE + 3T'] detected.
[2022-08-03 11:41:16,397] 253/1769 recon-all results matching ['Default'] detected.


Select only runs from subjects with multiple scans that have been preprocessed with all registered execution configurations:

In [4]:
unstacked_results = results.unstack()
stacked_results = results.clean.stack(["Hemisphere", "Region Name"])

In [5]:
NON_DEFAULT_CONFIGURATIONS = [ec for ec in results.configuration if ec != ExecutionConfiguration.DEFAULT]
NON_DEFAULT_CONFIGURATION_NAMES = [ec.value for ec in NON_DEFAULT_CONFIGURATIONS]
HEMISPHERES = stacked_results.index.unique(level="Hemisphere")
REGION_NAMES = stacked_results.index.unique(level="Region Name")
METRICS = stacked_results.columns
KS_RESULTS = "Statistic", "p-value"
SIGNIFICANT_METRICS = "Average Thickness", "Thickness StdDev", "Gray Matter Volume"

In [6]:
pairplot_destination = "average_thickness_pairplot.png"
_ = results.plot_mean_pairplot(destination=pairplot_destination)

AttributeError: 'str' object has no attribute 'exists'

In [ ]:
pairplot_destination = "gmv_pairplot.png"
_ = results.plot_mean_pairplot(metric="Gray Matter Volume", destination=pairplot_destination)

## KS Test

Calculate both whole-brain and region-wise KS statistics for the raw values:

In [ ]:
default_results = results.filter_by_configuration(ExecutionConfiguration.DEFAULT).stack(["Hemisphere", "Region Name"])
ks_whole_brain_index = pd.MultiIndex.from_product([NON_DEFAULT_CONFIGURATION_NAMES, METRICS], 
                                                  names=["Configuration", "Metric"])
ks_whole_brain = pd.DataFrame(index=ks_whole_brain_index, columns=KS_RESULTS)
ks_region_wise_index = pd.MultiIndex.from_product([NON_DEFAULT_CONFIGURATION_NAMES, HEMISPHERES, REGION_NAMES, METRICS], 
                                                  names=["Configuration", "Hemisphere", "Region Name", "Metric"])
ks_region_wise = pd.DataFrame(index=ks_region_wise_index, columns=KS_RESULTS).sort_index()
for ec in NON_DEFAULT_CONFIGURATIONS:
    ec_mask = results.context["Configuration"] == ec.value
    ec_run_ids = results.context.loc[ec_mask].index
    ec_results = stacked_results.loc[ec_run_ids]
    for metric in METRICS:
        ec_whole_brain_metric_values = ec_results[metric]
        ks_result = ks_2samp(ec_results[metric], default_results[metric])
        ks_whole_brain.loc[(ec.value, metric), :] = [ks_result.statistic, ks_result.pvalue]
        for hemisphere in HEMISPHERES:
            for region_name in REGION_NAMES:
                default_region_results = default_results.xs((hemisphere, region_name), 
                                                             level=("Hemisphere", "Region Name"))
                ec_region_results = ec_results.xs((hemisphere, region_name), 
                                                   level=("Hemisphere", "Region Name"))
                ks_result = ks_2samp(ec_region_results[metric], default_region_results[metric])
                ks_region_wise.loc[(ec.value, hemisphere, region_name, metric), :] = [ks_result.statistic, ks_result.pvalue]

### Correct for multiple comparisons

In [ ]:
ALPHA = 0.05
METHOD = "bonferroni"
CORRECTION_CONFIGURATION = {"alpha": ALPHA, "method": METHOD}

_, ks_whole_brain["p-value (corrected)"], _, _ = multipletests(ks_whole_brain["p-value"], **CORRECTION_CONFIGURATION)
_, ks_region_wise["p-value (corrected)"], _, _ = multipletests(ks_region_wise["p-value"], **CORRECTION_CONFIGURATION)

### Results

In [ ]:
whole_brain_significant = ks_whole_brain["p-value (corrected)"] < ALPHA
ks_whole_brain[whole_brain_significant].sort_index()

In [ ]:
ks_whole_brain[whole_brain_significant].sort_index().to_csv("whole-brain KS.csv")

In [ ]:
ks_region_wise

In [ ]:
region_wise_significant = ks_region_wise["p-value (corrected)"] < ALPHA
ks_region_wise[region_wise_significant].sort_index()
# ks_region_wise[region_wise_significant].xs("With T2", level="Configuration")

In [ ]:
ks_region_wise[region_wise_significant].sort_index().to_csv("region-wise KS.csv")

## Plot results

In [ ]:
diff = ReconAllDifferences(results=results)

In [ ]:
diff.create_difference_of_means_niftis()

In [ ]:
from surfer import Brain, project_volume_data
from mayavi import mlab
from pathlib import Path
import os

SURFACE_REGISTRATION = Path(os.environ["FREESURFER_HOME"]) / "average/mni152.register.dat"
in_file = "/home/zvi/Projects/brainprint/src/brainprint/recon_all/exports/differences/regional means/nii/T2_vs_Default_Average Thickness.nii.gz"

surf_data_lh = np.nan_to_num(project_volume_data(in_file, "lh", str(SURFACE_REGISTRATION)))
surf_data_rh = np.nan_to_num(project_volume_data(in_file, "rh", str(SURFACE_REGISTRATION)))

fig = [mlab.figure(size=(1200, 1200)) for i in range(4)]
brain = Brain("fsaverage", "split", "pial", views=["lat", "med"], background="white", figure=fig)
brain.add_data(surf_data_lh, hemi="lh", colormap="RdYlGn")
brain.add_data(surf_data_rh, hemi="rh", colormap="RdYlGn")

vmax = np.nanmax(np.abs(nib.load(in_file).get_fdata()))
brain.scale_data_colormap(0,  vmax/2, vmax, transparent=True, center=0)

brain.save_image("test.png", mode="rgba")

In [ ]:
mlab.close(all=True)

In [ ]:
T2_THICKNESS_PATH = "/home/zvi/Projects/brainprint/src/brainprint/recon_all/exports/differences/regional means/nii/T2_vs_DEFAULT_Average Thickness.nii.gz"
t2_thickness = nib.load(T2_THICKNESS_PATH).get_fdata()
vmax = np.nanmax(np.abs(t2_thickness))

In [ ]:
plot_img_on_surf(T2_THICKNESS_PATH, inflate=True, cmap="RdYlGn", vmax=vmax, output_file="T2 thickness diff.png", title="Difference in Mean Average Thickness Between T2 Workflow and Default")

In [ ]:
import matplotlib.colors as colors

# Nilearn Destrieux region name with no represenation in FreeSurfer
MEDIAL_WALL: str = "Medial_wall"

def plot_destrieux_scores(
    scores: pd.DataFrame,
    hemisphere: str = "Left",
    metric: str = "Surface Area",
    title: str = None,
    symmetric_cmap: bool = False,
    cmap: str = None,
    vmin: float = None,
    vmax: float = None,
    factor: int = 1,
    null_value: float = 0,
    serialize: bool = True,
) -> plt.Figure:
    destrieux_atlas = datasets.fetch_atlas_surf_destrieux()
    fsaverage = datasets.fetch_surf_fsaverage()
    destrieux_labels = [
        label.decode() for label in destrieux_atlas["labels"][1:]
    ]
    data = scores.xs(hemisphere, level="Hemisphere", axis=0)
    cmap = cmap if cmap is not None else "RdYlGn" if symmetric_cmap else "autumn"
    destrieux_projection = destrieux_atlas[f"map_{hemisphere.lower()}"].copy()
    region_ids = sorted(set(destrieux_projection))
    for i, region_id in enumerate(region_ids):
        label = destrieux_labels[i]
        if label == MEDIAL_WALL:
            value = null_value
        else:
            try:
                value = data.loc[label, metric]
            except KeyError:
                value = null_value
        region_mask = destrieux_projection == region_id
        destrieux_projection[region_mask] = value * factor
    vmin = destrieux_projection.min() if vmin is None else vmin
    vmax = destrieux_projection.max() if vmax is None else vmax
    surface = plotting.view_surf(
        fsaverage[f"infl_{hemisphere.lower()}"],
        destrieux_projection,
        bg_map=fsaverage[f"sulc_{hemisphere.lower()}"],
        cmap=cmap,
        # title=title,
        symmetric_cmap=symmetric_cmap,
        vmin=vmin,
        vmax=vmax,
    )
    surface.resize(900, 600)
    if serialize:
        np.save("tmp.npy", destrieux_projection)
    return surface

In [ ]:

import nibabel as nib
import pandas as pd
from nilearn import datasets

ATLAS_FETCHERS = {
    "destrieux": datasets.fetch_atlas_destrieux_2009,
}

In [ ]:
atlas = "destrieux"
fetcher = ATLAS_FETCHERS[atlas]
atlas_dict = fetcher()
atlas_nii_path, atlas_labels = atlas_dict["maps"], atlas_dict["labels"]
atlas_nii = nib.load(atlas_nii_path)
atlas_data = atlas_nii.get_fdata()

In [ ]:
metric = "Average Thickness"
# metric = "Thickness StdDev"
# metric = "Gray Matter Volume"

In [ ]:
plot_destrieux_scores(mean_differences[ExecutionConfiguration.FLAIR], 
                      metric=metric,
                      hemisphere="Left",
                      symmetric_cmap=True,
                      vmax=mean_differences[ExecutionConfiguration.FLAIR][metric].abs().max() * 10,
                      factor=10)

In [ ]:
plot_destrieux_scores(mean_differences[ExecutionConfiguration.FLAIR.value], 
                      metric=metric,
                      hemisphere="Right",
                      symmetric_cmap=True,
                      vmax=mean_differences[configuration][metric].abs().max() * 10,
                      factor=10)

In [ ]:
plot_destrieux_scores(mean_differences[ExecutionConfiguration.T2.value], 
                      metric=metric,
                      hemisphere="Left",
                      symmetric_cmap=True,
                      vmax=mean_differences[configuration][metric].abs().max() * 10,
                      factor=10)

In [ ]:
plot_destrieux_scores(mean_differences[ExecutionConfiguration.T2.value], 
                      metric=metric,
                      hemisphere="Right",
                      symmetric_cmap=True,
                      vmax=mean_differences[configuration][metric].abs().max() * 10,
                      factor=10)

## Intra-class Correlation

In [ ]:
unstacked_results["Configuration"] = unstacked_results.apply(lambda row: multi_context.loc[row["Run ID"], "Configuration"], axis=1)

In [ ]:
icc = pg.intraclass_corr(data=avg_thickness_long, targets="Lateral Region Name", raters="Configuration", ratings="Value")

In [ ]:
icc